# StrangleBot
We are building a trading bot to execute short strangles and long strangles on S&P 500 companies based on features we can extract from yfinance data.

## Setup

`git clone https://github.com/PeterFavero/cqf_final_project.git`

`cd cqf_final_project`

`python3 -m venv venv`

MacOS/Linux: `source venv/bin/activate` or Windows: `venv\Scripts\activate`

`pip3 install -r requirements.txt`

In [50]:
# IMPORTS
# ---------------------------------------------
import yfinance as yf
import pandas as pd
import pytz
import ssl
from datetime import datetime, timedelta
from tqdm import tqdm
import csv
import sys
import os
import requests
from io import StringIO

In [51]:
# ENVIROMENT CONFIGURATION
# ---------------------------------------------
# This is nescessary for web scraping
ssl._create_default_https_context = ssl._create_unverified_context

In [52]:
# CONSTANTS
# ---------------------------------------------
SP500_TICKER = "^GSPC"
TEN_YEARS_AGO = pd.Timestamp(datetime.now() - timedelta(days=365*10)).replace(tzinfo=pytz.UTC)
ORIGINAL_STDOUT = sys.stdout
SUPPRESSED_STDOUT = open(os.devnull, 'w')

In [53]:
# DATA: HELPER FUNCTIONS 
# ---------------------------------------------

def parse_nasdaqlist():
    output_array = []  # Array to hold the extracted parts of lines
    with open('data/nasdaqlisted.txt', 'r') as file:  # Open the file in read mode
        next(file)  # Skip the first line
        for line in file:  # Iterate through each line in the file after the first
            # Find the position of the first '|' character
            separator_index = line.find('|')
            if separator_index != -1:  # If '|' is found
                # Extract the part of the line before the '|' character
                substring = line[:separator_index].strip()
                output_array.append(substring)  # Append to the output array
            else:
                # Handle cases where there is no '|'
                output_array.append(line.strip())
    return output_array

def ticker_listing_age(ticker, hist_dict) : 
    hist = hist_dict[ticker]
    return hist.index[0]

def data_of_tickers(full_ticker_strings_list) :
    cleaned_ticker_strings_list =  [ticker for ticker in full_ticker_strings_list if "." not in ticker]
    cleaned_ticker_objects_dict = {}
    cleaned_ticker_hists_dict = {}
    delisted_ticker_strings_list = []
    for ticker in tqdm(cleaned_ticker_strings_list, desc="Downloading price history data") :
        try :
            cleaned_ticker_objects_dict[ticker] = yf.Ticker(ticker)
            cleaned_ticker_hists_dict[ticker] = cleaned_ticker_objects_dict[ticker].history(period='2y')
        except Exception as _ :
            delisted_ticker_strings_list.appened(ticker)
    return cleaned_ticker_strings_list, cleaned_ticker_objects_dict, cleaned_ticker_hists_dict, delisted_ticker_strings_list


In [54]:
# Data: Tickers list
# ---------------------------------------------
sp500_unprocessed = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_list()
nasdaq_unprocessed = parse_nasdaqlist()
sp500_and_nasdaq_unprocessed = list( set(sp500_unprocessed) | set(nasdaq_unprocessed) )

['AACG', 'AACI', 'AACIU', 'AACIW', 'AADI', 'AADR', 'AAGR', 'AAGRW', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAXJ', 'ABAT', 'ABCB', 'ABCL', 'ABCS', 'ABEO', 'ABIO', 'ABL', 'ABLLL', 'ABLLW', 'ABLV', 'ABLVW', 'ABNB', 'ABOS', 'ABSI', 'ABTS', 'ABUS', 'ABVC', 'ABVX', 'ACAB', 'ACABU', 'ACABW', 'ACAC', 'ACACU', 'ACACW', 'ACAD', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACCD', 'ACDC', 'ACET', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACHL', 'ACHV', 'ACIC', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACMR', 'ACNB', 'ACNT', 'ACON', 'ACONW', 'ACRS', 'ACRV', 'ACST', 'ACT', 'ACTG', 'ACVA', 'ACWI', 'ACWX', 'ACXP', 'ADAG', 'ADAP', 'ADBE', 'ADD', 'ADEA', 'ADI', 'ADIL', 'ADMA', 'ADN', 'ADNWW', 'ADP', 'ADPT', 'ADSE', 'ADSEW', 'ADSK', 'ADTH', 'ADTHW', 'ADTN', 'ADTX', 'ADUS', 'ADV', 'ADVM', 'ADVWW', 'ADXN', 'AEAE', 'AEAEU', 'AEAEW', 'AEHL', 'AEHR', 'AEI', 'AEIS', 'AEMD', 'AENT', 'AENTW', 'AEP', 'AERT', 'AERTW', 'AEYE', 'AEZS', 'AFAR', 'AFARU', 'AFARW', 'AFBI', 'AFCG', 'AFIB', 'AFJK', 'AFJKR', 'AFJKU', 'AFMD',

In [55]:
ticker_strings_list, ticker_objects_dict, ticker_hists_dict, delisted_ticker_strings_list = data_of_tickers(sp500_and_nasdaq_unprocessed)

KeyboardInterrupt: 